## Goroutines and Channels

Go enables two styles of concurrent programming. This chapter presents goroutines and channels, which support communicating sequential processes or CSP, a model of concurrency in which values are passed between independent activities (goroutines) but variables are for the most part confined to a single activity.

In Go, each concurrently executing activity is called a goroutine. Consider a program that has two functions, one that does some computation and one that writes some output, and assume that neither function calls the other. A sequential program may call one function and then call the other, but in a concurrent program with two or more goroutines, calls to both functions can be active at the same time.

In [1]:
// spinner
import (
    "fmt"
    "time"
)

func main() {
    go spinner(100 * time.Millisecond)
    const n = 35
    fibN := fib(n) // slow
    fmt.Printf("\rFibonacci(%d) = %d\n", n, fibN)
}

func spinner(delay time.Duration) {
    for {
        for _, r := range `-\|/` {
            fmt.Printf("\r%c", r)
            time.Sleep(delay)
        }
    }
}

func fib(x int) int {
    if x < 2 {
        return x
    }
    return fib(x-1) + fib(x-2)
}

main()

Fibonacci(35) = 9227465


## Channels
If goroutines are the activities of a concurrent Go program, channels are the connections between them.

Channels are a typed conduit through which you can send and receive values with the channel operator, <-

In [2]:
func sum(s []int, c chan int) {
    sum := 0
    for _, v := range s {
        sum += v
    }
    c <- sum // send sum to c
}

s := []int{7, 2, 8, -9, 4, 0}

c := make(chan int)
go sum(s[:len(s)/2], c)
go sum(s[len(s)/2:], c)
x, y := <-c, <-c // receive from c
fmt.Println(x, y, x + y)

-5 17 12


9 <nil>

A channel has two principal operations, send and receive, collectively known as communications. A send statement transmits a value from one goroutine, through the channel, to another goroutine executing a corresponding receive expression. Both operations are written using the <- operator.

In [3]:
func calc(i int, c chan int) {
    c <- 42 + i
}

In [4]:
nums := []int{1, 2, 3, 4, 5}
c := make(chan int)

// send them all for calculation
for i:= 0; i<len(nums); i++ {
    go calc(nums[i], c)
}

// get the results back
for i:= 0; i<len(nums); i++ {
    result := <- c
    fmt.Println(result)
}

47
43
44
45
46


Notice that you won't necessarily get results back in the order they were sent.

Channels can be buffered. Provide the buffer length as the second argument to make to initialize a buffered channel.

In [5]:
ch := make(chan int, 100)

In [6]:
type Square struct {
    Width int
}

func (s Square) Area() int {
    return s.Width * s.Width
}

In [7]:
s := Square{2}
s.Area()

4

In [8]:
squares := []Square{
    Square{1},
    Square{2},
    Square{3},
}

squares

[{1} {2} {3}]

In [9]:
areas := make(map[Square]int)

func recordArea(s Square) {
    areas[s] = s.Area()
}

for _, s := range squares {
    recordArea(s)
}

In [10]:
areas

map[{1}:1 {2}:4 {3}:9]

We can make this concurrent...

In [5]:
type Square struct {
    Width int
}

func (s Square) Area() int {
    return s.Width * s.Width
}

In [9]:
// see the results with conc.go
import "fmt"

func recordArea(s Square, areas map[Square]int, c chan bool) {
    areas[s] = s.Area()
    c <- true
}

func main() {
    squares := []Square{
        Square{1},
        Square{2},
        Square{3},
    }

    areas := make(map[Square]int)
    c := make(chan bool)

    for _, s := range squares {
        go recordArea(s, areas, c)
    }

    for i := 0; i < len(squares); i++ {
        <-c
    }

    fmt.Println(areas)
}


## Combining this with select